In [28]:
# Import Dependencies
import os
import pandas as pd
from pathlib import Path

In [29]:
kaggle_path = Path("data/US_Accidents_March23.csv")
kaggle_df = pd.read_csv(kaggle_path)
kaggle_df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [31]:
kaggle_df.info()
#<!-- Source, End_Lat, End_Lng, Distance(mi), Country, Weather_Timestamp, Civil_Twilight, Astronomical_Twilight -->

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)       

In [32]:
#Columns to delete:
#Source, End_Lat, End_Lng, Distance(mi), Country, Weather_Timestamp, Civil_Twilight, Astronomical_Twilight
kaggle_df_copy = kaggle_df.copy()

columns_to_delete = [
    'Source', 'End_Lat', 'End_Lng', 'Distance(mi)', 
    'Country', 'Weather_Timestamp', 'Civil_Twilight', 'Astronomical_Twilight'
]

kaggle_df_copy = kaggle_df_copy.drop(columns=columns_to_delete)
kaggle_df_copy.head()


,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Description,Street,City,County,...,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Nautical_Twilight
0,A-1,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,Right lane blocked due to accident on I-70 Eas...,I-70 E,Dayton,Montgomery,...,False,False,False,False,False,False,False,False,Night,Night
1,A-2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,Accident on Brice Rd at Tussing Rd. Expect del...,Brice Rd,Reynoldsburg,Franklin,...,False,False,False,False,False,False,False,False,Night,Night
2,A-3,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,Williamsburg,Clermont,...,False,False,False,False,False,False,True,False,Night,Day
3,A-4,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,Dayton,Montgomery,...,False,False,False,False,False,False,False,False,Night,Day
4,A-5,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,Dayton,Montgomery,...,False,False,False,False,False,False,True,False,Day,Day


In [33]:
kaggle_df_cleaned = kaggle_df.dropna()

# Display the first few rows of the cleaned DataFrame
kaggle_df_cleaned.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
3402762,A-3412645,Source1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,...,False,False,False,False,False,False,Night,Night,Night,Night
3402767,A-3412650,Source1,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,...,False,False,False,False,False,False,Day,Day,Day,Day
3402771,A-3412654,Source1,2,2016-02-08 11:51:46,2016-02-08 17:51:46,41.375310,-81.820170,41.367860,-81.821740,0.521,...,False,False,False,False,False,False,Day,Day,Day,Day
3402773,A-3412656,Source1,2,2016-02-08 15:16:43,2016-02-08 21:16:43,40.109310,-82.968490,40.110780,-82.984000,0.826,...,False,False,False,False,False,False,Day,Day,Day,Day
3402774,A-3412657,Source1,2,2016-02-08 15:43:50,2016-02-08 21:43:50,39.192880,-84.477230,39.196150,-84.473350,0.307,...,False,False,False,False,False,False,Day,Day,Day,Day


In [36]:
roundabout_counts = kaggle_df_cleaned["Roundabout"].value_counts()
print(roundabout_counts)

Roundabout
False    3554440
True         109
Name: count, dtype: int64


In [54]:
#Convert Start_Time column to date/time format
kaggle_df_cleaned = kaggle_df_cleaned.copy()
kaggle_df_cleaned.loc[:, 'Start_Time'] = pd.to_datetime(kaggle_df_cleaned['Start_Time'])
# Convert Start_Time to datetime format if it's not already
kaggle_df_cleaned.loc[:, 'Start_Time'] = pd.to_datetime(kaggle_df_cleaned['Start_Time'])

# Extract date without time and create a new column
kaggle_df_cleaned.loc[:, 'Date'] = kaggle_df_cleaned['Start_Time'].dt.date

# Extract day of the week and create another new column
kaggle_df_cleaned.loc[:, 'Day_of_Week'] = kaggle_df_cleaned['Start_Time'].dt.day_name()
kaggle_df_cleaned.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Day_of_Week,Date
3402762,A-3412645,Source1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,...,False,False,False,False,Night,Night,Night,Night,Monday,2016-02-08
3402767,A-3412650,Source1,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,...,False,False,False,False,Day,Day,Day,Day,Monday,2016-02-08
3402771,A-3412654,Source1,2,2016-02-08 11:51:46,2016-02-08 17:51:46,41.375310,-81.820170,41.367860,-81.821740,0.521,...,False,False,False,False,Day,Day,Day,Day,Monday,2016-02-08
3402773,A-3412656,Source1,2,2016-02-08 15:16:43,2016-02-08 21:16:43,40.109310,-82.968490,40.110780,-82.984000,0.826,...,False,False,False,False,Day,Day,Day,Day,Monday,2016-02-08
3402774,A-3412657,Source1,2,2016-02-08 15:43:50,2016-02-08 21:43:50,39.192880,-84.477230,39.196150,-84.473350,0.307,...,False,False,False,False,Day,Day,Day,Day,Monday,2016-02-08


In [ ]:
#What days of the week accidents occur the most?
# Group by 'Day_of_Week' and count occurrences
day_counts = kaggle_df_cleaned.groupby('Day_of_Week').size().reset_index(name='Count')

# Sort the results by day of the week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_counts['Day_of_Week'] = pd.Categorical(day_counts['Day_of_Week'], categories=day_order, ordered=True)
day_counts = day_counts.sort_values('Day_of_Week')

print(day_counts)

  Day_of_Week   Count
1      Monday  501906
5     Tuesday  540262
6   Wednesday  558927
4    Thursday  573657
0      Friday  613030
2    Saturday  423522
3      Sunday  343245


In [ ]:
#Do roundabouts help with reducing accidents?
total_accidents = kaggle_df_cleaned.shape[0]
roundabout_accidents = kaggle_df_cleaned["Roundabout"].sum()
percentage_roundabout = (roundabout_accidents / total_accidents) * 100
percentage_no_roundabout = ((total_accidents - roundabout_accidents)/total_accidents)*100
print (f"Percentage of accidents with roundabouts is: {percentage_roundabout:.2f}%")
print (f"Percentage of accidents without roundabouts is: {percentage_no_roundabout:.2f}%")


Percentage of accidents with roundabouts is: 0.00%
Percentage of accidents without roundabouts is: 100.00%


In [17]:

#Read ACC_AUX.csv
ACC_AUX_path = Path("data/ACC_AUX.csv")

# Import to data frame
ACC_AUX_df = pd.read_csv(ACC_AUX_path)

# Display data frame
ACC_AUX_df.head()

,YEAR,STATE,ST_CASE,FATALS,A_CRAINJ,A_REGION,A_RU,A_INTER,A_RELRD,A_INTSEC,...,A_RD,A_HR,A_DIST,A_DROWSY,A_WRONGWAY,BIA,SPJ_INDIAN,INDIAN_RES,CENSUS_2020_TRACT_FIPS,TRACT
0,2022,1,10001,1,1,4,1,2,1,2,...,2,2,2,2,2,0,0,0,1.107050e+09,1
1,2022,1,10002,2,1,4,1,2,1,2,...,2,2,2,2,1,0,0,0,1.101006e+09,1
2,2022,1,10003,1,1,4,1,2,4,2,...,1,2,2,2,2,0,0,0,1.115040e+09,1
3,2022,1,10004,1,1,4,1,1,4,2,...,1,2,2,2,2,0,0,0,1.101006e+09,1
4,2022,1,10005,1,1,4,2,1,2,2,...,1,1,2,2,2,0,0,0,1.073013e+09,1


In [18]:
#Read accident.csv
accident_path = Path("data/accident.csv")

# Import to data frame
accident_df = pd.read_csv(accident_path)

# Display data frame
accident_df.head()

,STATE,STATENAME,ST_CASE,PEDS,PERNOTMVIT,VE_TOTAL,VE_FORMS,PVH_INVL,PERSONS,PERMVIT,...,NOT_MINNAME,ARR_HOUR,ARR_HOURNAME,ARR_MIN,ARR_MINNAME,HOSP_HR,HOSP_HRNAME,HOSP_MN,HOSP_MNNAME,FATALS
0,1,Alabama,10001,0,0,2,2,0,3,3,...,47,13,1:00pm-1:59pm,4,4,13,1:00pm-1:59pm,47,47,1
1,1,Alabama,10002,0,0,2,2,0,5,5,...,Unknown,99,Unknown EMS Scene Arrival Hour,99,Unknown EMS Scene Arrival Minutes,99,Unknown,99,Unknown EMS Hospital Arrival Time,2
2,1,Alabama,10003,0,0,1,1,0,2,2,...,33,1,1:00am-1:59am,50,50,99,Unknown,99,Unknown EMS Hospital Arrival Time,1
3,1,Alabama,10004,0,0,1,1,0,1,1,...,48,15,3:00pm-3:59pm,9,9,15,3:00pm-3:59pm,44,44,1
4,1,Alabama,10005,1,1,1,1,0,1,1,...,48,18,6:00pm-6:59pm,54,54,88,Not Applicable (Not Transported),88,Not Applicable (Not Transported),1


In [19]:
#Read distract.csv
#Different distractions
distract_path = Path("data/distract.csv")

# Import to data frame
distract_df = pd.read_csv(distract_path)

# Display data frame
distract_df.head()

,STATE,STATENAME,ST_CASE,VEH_NO,DRDISTRACT,DRDISTRACTNAME
0,1,Alabama,10001,1,96,Not Reported
1,1,Alabama,10001,2,96,Not Reported
2,1,Alabama,10002,1,96,Not Reported
3,1,Alabama,10002,2,96,Not Reported
4,1,Alabama,10003,1,96,Not Reported


In [20]:
#Read weather.csv
#How weather affects accidents
weather_path = Path("data/weather.csv")

# Import to data frame
weather_df = pd.read_csv(weather_path)

# Display data frame
weather_df.head()

,STATE,STATENAME,ST_CASE,WEATHER,WEATHERNAME
0,1,Alabama,10001,1,Clear
1,1,Alabama,10002,1,Clear
2,1,Alabama,10003,10,Cloudy
3,1,Alabama,10004,10,Cloudy
4,1,Alabama,10005,2,Rain
